In [ ]:
import json
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from folium.plugins import HeatMap
import folium as fl
import datetime
sns.set(style="whitegrid")
%matplotlib inline
from ipywidgets import *
from ipywidgets import interact, interactive, fixed
from IPython.display import display
#from IPython.html import widgets
import ipywidgets as widgets



In [ ]:
battles = pd.DataFrame([json.loads(line) for line in open("../datasets/battle-features-0.json")])
btp = battles[['start_date','casualties_1', 'casualties_2', 'dates_bc', 'end_date']]
btp = btp[pd.notnull(btp['start_date'])]
btp['casualties_total']=btp['casualties_1']+btp['casualties_2']

btpbc=btp[btp.dates_bc==True]
btpbc['year'] = [-datetime.datetime.strptime(date, "%Y-%m-%d").year for date in btpbc['start_date']]
btpnbc=btp[btp.dates_bc==False]
btpnbc['year'] = [datetime.datetime.strptime(date, "%Y-%m-%d").year for date in btpnbc['start_date']]
btp=btpbc.append(btpnbc)

bins = [-2500,-2250,-2000,-1750,-1500,-1250,-1000,-750,-500,-250,0,250,500,750,1000,1250,1500,1750,2000,2250]
bins_labels=[-2500,-2250,-2000,-1750,-1500,-1250,-1000,-750,-500,-250,0,250,500,750,1000,1250,1500,1750,2000]
casualtiesbins = [0,50,100,250, 500,1000,2000,5000,10000,50000,1000000,4000000,5000000]
casualties_labels=[0,50,100,250, 500,1000,2000,5000,10000,50000,1000000,4000000]
lengthbins = [-1,5,10,15,50,100,150,200,500,1000,2000,3000]
lengthbins_labels=[0,5,10,15,50,100,150,200,500,1000,2000]

In [ ]:
btpcasualties=btp.groupby(pd.cut(btp['casualties_total'], bins=casualtiesbins)).casualties_total
casualties_count = btpcasualties.count()
casualties_count.plot(kind='bar')

In [ ]:
casualties_count.plot(kind='box')

In [ ]:
btpDP = btp.groupby(pd.cut(btp['year'],bins=bins, labels=bins_labels))[['start_date','casualties_total', 'end_date']]
  
def precomputation_c(b):
        btpnow = btpDP.get_group(b)
        casualties=btpnow.groupby(pd.cut(btpnow['casualties_total'], bins=casualtiesbins, labels=casualties_labels))['casualties_total']
        casualties_count = casualties.count()
        return casualties_count
    
def plotCasualtiesByPeriod(b):
    try:
        casualties_count = precomputation_c(b)
        plt.figure(1)
        casualties_count.plot(kind='bar')
        plt.title('casualties by time interval')
    except:
        print("No battles in time interval")
        
def plotCasualtiesStatsByPeriod(b):
    try:
        casualties_count = precomputation_c(b)
        plt.figure(1)
        casualties_count.plot(kind='box')
        plt.title('casualties by time interval')
    except:
        print("No battles in time interval")



In [ ]:
interact(plotCasualtiesByPeriod, b=(-2500,2000,250))

In [ ]:
interact(plotCasualtiesStatsByPeriod, b=(-2500,2000,250))

In [ ]:

btp['end_date']=btp['end_date'].fillna(btp['start_date'])
btp['start_date'] = btp['start_date'].astype('datetime64[ns]')
btp['end_date'] = btp['end_date'].astype('datetime64[ns]')
length=[]
btp=btp.reset_index()
for i in range(0,len(btp['start_date'])):
    #print(pd.Timedelta(btp1.end_date[i]-btp1.start_date[i]))
    length+=[pd.Timedelta(btp.end_date[i]-btp.start_date[i])]
btp['length'] = length
btp['length']=btp['length'].astype('timedelta64[D]')

btpCL = btp.groupby(pd.cut(btp['length'],bins=bins, labels=bins_labels))[['length','casualties_total']]

def precomputation_cl(b):
    btpnow = btpCL.get_group(b)
    casualties=btpnow.groupby(pd.cut(btpnow['casualties_total'], bins=casualtiesbins, labels=casualties_labels))['casualties_total']
    casualties_count = casualties.count()
    return casualties_count
  
def plotCasualtiesByLength(b):
    try:
        casualties_count = precomputation_cl(b)
        plt.figure(1)
        casualties_count.plot(kind='bar')
        plt.title('casualties by battle length interval')
    except:
        print("No battle casualties in length interval")

def plotCasualtiesByLengthStats(b):
    try:
        casualties_count = precomputation_cl(b)
        plt.figure(1)
        casualties_count.plot(kind='box')
        plt.title('casualties by battle length interval')
    except:
        print("No battle casualties in length interval")


In [ ]:
interact(plotCasualtiesByLength, b=lengthbins_labels)

In [ ]:
interact(plotCasualtiesByLengthStats, b=lengthbins_labels)